## Library Imports

In [1]:
from time import time
notebook_start_time = time()

In [2]:
import os
import re
import random as r
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torch.nn.utils import weight_norm as WN
from torchvision import models, transforms

import imgaug
import random as r
from imgaug import augmenters

import warnings
warnings.filterwarnings("ignore")

## Constants and Utilities

In [3]:
def breaker(num=50, char="*") -> None:
    print("\n" + num*char + "\n")


def head(x, no_of_ele=5) -> None:
    print(x[:no_of_ele])

    
def get_augment(seed: int):
    imgaug.seed(seed)
    augment = augmenters.SomeOf(None, [
        augmenters.HorizontalFlip(p=0.5),
        augmenters.VerticalFlip(p=0.5),
        augmenters.Affine(scale=(0.75, 1.25), translate_percent=(-0.1, 0.1), rotate=(-45, 45), seed=seed),
    ], seed=seed)

    return augment

    
def show(image: np.ndarray) -> None:
    plt.figure(figsize=(9, 6))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

In [4]:
TRANSFORM = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225]),
                               ])
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 0
MODEL_NAME = "densenet169"
NUM_FEATURES = 1664

## Dataset Template

In [5]:
class DS(Dataset):
    def __init__(self, images=None, transform=None):
        self.images    = images
        self.transform = transform
        
    def __len__(self):
        return self.images.shape[0]
    
    def __getitem__(self, idx):
        return self.transform(self.images[idx])

## Build DataLoader

In [6]:
def build_dataloader(images: np.ndarray, transform=None):    
    data_setup = DS(images=images, transform=transform)
    data = DL(data_setup, batch_size=64, shuffle=False)
    
    return data

## Build Model

In [7]:
def build_model(pretrained=True):
    class ImageModel(nn.Module):
        def __init__(self, pretrained=False):
            super(ImageModel, self).__init__()

            self.features = models.densenet169(pretrained=pretrained, progress=True)
            if pretrained:
                self.freeze()
            self.features = nn.Sequential(*[*self.features.children()][:-1])
            self.features.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(1, 1)))
            self.features.add_module("Flatten", nn.Flatten())
        
        def freeze(self):
            for params in self.parameters():
                params.requires_grad = False

        def forward(self, x):
            return self.features(x)

    model = ImageModel(pretrained=pretrained)
    return model

## Acquire Features Helper

In [8]:
def get_features(model=None, dataloader=None, num_features=None):
    model.to(DEVICE)
    model.eval()

    y_pred = torch.zeros(1, num_features).to(DEVICE)
    for X in dataloader:
        X = X.to(DEVICE)
        with torch.no_grad():
            output = model(X)
        y_pred = torch.cat((y_pred, output.view(-1, num_features)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

## Obtain and Save Features

In [9]:
def save_features():
    images = np.load("../input/petfinder-pretrained-images-ccropped/Images_224x224.npy")
    
    r.seed(SEED)
    seeds = [r.randint(0, 99) for _ in range(20)]
    
    breaker()
    for seed in seeds:
        augment = get_augment(seed)
        images = augment(images=images)
        dataloader = build_dataloader(images=images, transform=TRANSFORM)

        start_time = time()
        model = build_model(pretrained=True)
        features = get_features(model, dataloader, num_features=NUM_FEATURES)
        np.save("./{}_features_seed_{}.npy".format(MODEL_NAME, seed), features)
        print("Seed {} -> {:.2f} minutes".format(seed, (time()-start_time)/60))

    breaker()

In [10]:
save_features()


**************************************************



Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


  0%|          | 0.00/54.7M [00:00<?, ?B/s]

Seed 49 -> 0.49 minutes
Seed 97 -> 0.35 minutes
Seed 53 -> 0.35 minutes
Seed 5 -> 0.35 minutes
Seed 33 -> 0.35 minutes
Seed 65 -> 0.35 minutes
Seed 62 -> 0.34 minutes
Seed 51 -> 0.35 minutes
Seed 38 -> 0.35 minutes
Seed 61 -> 0.35 minutes
Seed 45 -> 0.35 minutes
Seed 74 -> 0.35 minutes
Seed 27 -> 0.34 minutes
Seed 64 -> 0.34 minutes
Seed 17 -> 0.35 minutes
Seed 36 -> 0.34 minutes
Seed 17 -> 0.34 minutes
Seed 96 -> 0.34 minutes
Seed 12 -> 0.35 minutes
Seed 79 -> 0.35 minutes

**************************************************



In [11]:
breaker()
print("Notebook Run Time : {:.2f} minutes".format((time()-notebook_start_time)/60))
breaker()


**************************************************

Notebook Run Time : 11.94 minutes

**************************************************

